In [ ]:
!pip install transformers

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
file = '/content/ganjarPranowo-231207.csv'
data = pd.read_csv(file, delimiter=';')
data.sample(5)

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
1452,Thu Dec 07 13:22:16 +0000 2023,1732752387742531598,@Gus_Raharjo @ganjarpranowo Ganjar Tuntaskan I...,0,0,0,0,in,1657009552594984960,1732646210912296966,bocil0475857390,https://twitter.com/bocil0475857390/status/173...
835,Thu Dec 07 13:54:49 +0000 2023,1732760578081611832,"IKN menjadi simbol transformasi positif, diman...",0,0,0,0,in,1686590547744727040,1732760578081611832,yahyacmad,https://twitter.com/yahyacmad/status/173276057...
237,Thu Dec 07 14:19:34 +0000 2023,1732766806811504999,@alex_hunter146 @puanmaharani_ri @mohmahfudmd ...,0,0,0,0,in,1716424878117785600,1732731497856336023,EdohTiti90875,https://twitter.com/EdohTiti90875/status/17327...
30,Thu Dec 07 14:26:54 +0000 2023,1732768653148406033,@Aryprasetyo85 @ganjarpranowo Bersama pak Ganj...,0,0,0,0,in,1398117829656776704,1732649186011840949,Hestiayu2807,https://twitter.com/Hestiayu2807/status/173276...
1241,Thu Dec 07 13:34:03 +0000 2023,1732755351546781966,@alfatih212426 @Lopea13 @fajr46338454 @AnakKam...,0,1,0,0,in,1266873119987261441,1732112390144311761,Cocottetapir,https://twitter.com/Cocottetapir/status/173275...


In [ ]:
print(data.shape)

(1593, 12)


In [ ]:
from googletrans.client import Translator
translator = Translator ()
from transformers import pipeline
sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
data['label'] = data['full_text'].str.encode('ascii', 'ignore').apply(translator.translate, dest='en')
data['label'] = data['label'].apply(getattr, args=("text",))
data['label'] = data.apply(lambda x: clean_tweet (x['label']), axis=1)

In [ ]:
df = (
    data
    .assign(sentiment=lambda x: x['label'].apply(lambda s: sentiment_classifier(s)))
    .assign(
        label=lambda x: x['sentiment'].apply(lambda s: s[0]['label']),
        score=lambda x: x['sentiment'].apply(lambda s: s[0]['score'])
    )
)

In [ ]:
df.sample(20)

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url,label,sentiment,score
1542,Thu Dec 07 13:16:20 +0000 2023,1732750892636705182,"Ganjar Pranowo, optimis IKN akan jadi tempat b...",0,0,0,0,in,1683734852326293510,1732750892636705182,mhmmdfirman_,https://twitter.com/mhmmdfirman_/status/173275...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.997718453407...",0.997718
393,Thu Dec 07 14:14:17 +0000 2023,1732765476294402544,@jijindut @ganjarpranowo selalu melaksanakan a...,0,0,0,0,in,1639392274395328512,1732688810276962306,salsabilaa_tii,https://twitter.com/salsabilaa_tii/status/1732...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.998746156692...",0.998746
647,Thu Dec 07 14:03:13 +0000 2023,1732762690916090280,Keberhasilan Ganjar Pranowo dalam mengembangka...,0,0,0,0,in,1679747992721698816,1732762690916090280,anna_putriyani,https://twitter.com/anna_putriyani/status/1732...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.998311161994...",0.998311
944,Thu Dec 07 13:49:45 +0000 2023,1732759304623870176,"Sebagai capres 2024, Ganjar Pranowo dianggap m...",0,0,0,0,in,1681927627962208256,1732759304623870176,NurFaridaa_10,https://twitter.com/NurFaridaa_10/status/17327...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.989861845970...",0.989862
581,Thu Dec 07 14:06:54 +0000 2023,1732763619346006272,@Aryprasetyo85 @ganjarpranowo @007koteka @03__...,0,0,0,0,in,1603538645457457152,1732690536702910649,irwanka5,https://twitter.com/irwanka5/status/1732763619...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.992178261280...",0.992178
241,Thu Dec 07 14:19:29 +0000 2023,1732766784141316305,Setiap proses adalah langkah menuju kematangan...,0,0,0,0,in,1689484501259517952,1732766784141316305,MayaCita510246,https://twitter.com/MayaCita510246/status/1732...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.999413132667...",0.999413
465,Thu Dec 07 14:11:50 +0000 2023,1732764860750913552,"Melalui KTP SAKTI, Ganjar Pranowo menjembatani...",0,0,0,0,in,1691012239623606272,1732764860750913552,evitalistiana,https://twitter.com/evitalistiana/status/17327...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.948300004005...",0.948300
363,Thu Dec 07 14:15:04 +0000 2023,1732765675016667152,@ganjarpranowo karena agama ada lah tiangnya d...,0,0,0,0,in,1700041508022603776,1732589528039456768,tariaan8,https://twitter.com/tariaan8/status/1732765675...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.787628173828...",0.787628
583,Thu Dec 07 14:06:48 +0000 2023,1732763592145932467,"Kreatif dan optimis, itulah IKN! Ganjar Pranow...",0,0,0,0,in,1684847611235483648,1732763592145932467,MonikaAgnesss,https://twitter.com/MonikaAgnesss/status/17327...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.675330698490...",0.675331
540,Thu Dec 07 14:09:22 +0000 2023,1732764239272579277,Jawab Debat dg Teks -- #aniesbaswedan #ganjarp...,0,1,0,0,in,933533092853059586,1732764239272579277,KanalTerkini,https://twitter.com/KanalTerkini/status/173276...,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.986160337924...",0.986160


In [ ]:
simpan = df.to_csv('labeled_ganjar.csv', index=False)

In [ ]:
dt = pd.read_csv('labeled_ganjar.csv')

In [ ]:
dt.head(5)

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url,label,sentiment,score
0,Thu Dec 07 14:28:20 +0000 2023,1732769013707530623,Dukungan untuk Pak Ganjar Pranowo dan Pak Mahf...,0,0,0,0,in,1689896450145325056,1732769013707530623,RizalFauz56,https://twitter.com/RizalFauz56/status/1732769...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.996817231178...",0.996817
1,Thu Dec 07 14:28:20 +0000 2023,1732769011107033157,Keberhasilan Pak Ganjar Pranowo dan Pak Mahfud...,0,0,0,0,in,1690282016804384768,1732769011107033157,alvingunawann71,https://twitter.com/alvingunawann71/status/173...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.999148607254...",0.999149
2,Thu Dec 07 14:28:17 +0000 2023,1732769000659009608,@Gus_Raharjo @ganjarpranowo Sudah ada dalam pr...,0,0,0,0,in,1698968732138377216,1732646210912296966,ganjaranku,https://twitter.com/ganjaranku/status/17327690...,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.995126247406...",0.995126
3,Thu Dec 07 14:28:17 +0000 2023,1732768999664918668,Kepercayaan masyarakat Jatim terhadap Pak Ganj...,0,0,0,0,in,1688836931558551552,1732768999664918668,fadilladin25,https://twitter.com/fadilladin25/status/173276...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.992251098155...",0.992251
4,Thu Dec 07 14:28:15 +0000 2023,1732768992484344306,"KTP SAKTI bukan hanya kartu, tapi juga lambang...",0,0,0,0,in,1687399634300538880,1732768992484344306,ragaangkasaaaa,https://twitter.com/ragaangkasaaaa/status/1732...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.714086472988...",0.714086
